In [4]:
import numpy as np;
import wave;

In [215]:
ARRAY_TYPE = {
    1: np.int8,
    2: np.int16,
    4: np.int32,
}

def applyMagic(srcfile, coefficient = 1.3, batchSize = 128):
    with wave.open(srcfile, 'rb') as src:
        sampleRate = src.getframerate();
        channels = src.getnchannels();
        sampleWidth = src.getsampwidth();
        totalFrames = src.getnframes();
        frames = src.readframes(totalFrames);
        
    sound = np.frombuffer(frames, dtype=ARRAY_TYPE[sampleWidth]).reshape(-1, channels);
    
    resultList = [];
    for i in range(channels):
        resultList.append(changeFreq(sound[:, i], coefficient, batchSize));
    
    sound = np.column_stack(tuple(resultList));
    
    with wave.open('output_' + srcfile, 'wb') as dest:
        dest.setnchannels(channels);
        dest.setsampwidth(sampleWidth);
        dest.setframerate(sampleRate);
        dest.writeframes(sound.tobytes());
        
    print('Файл записан!');
    
def changeFreq(arr, coefficient, batchSize):
    overlap = int(batchSize * 0.775); # Имперический минимум артефактов
    window = np.hanning(batchSize);  # Говорят, это сгладит дорожку
    result = np.zeros_like(arr, dtype=np.float32);

    for i in range(0, len(arr) - batchSize + 1, batchSize - overlap):
        temp = arr[i:i + batchSize] * window;
        temp = np.fft.rfft(temp);
        
        freqChanged = np.zeros_like(temp, dtype=np.complex64);
        for j in range(len(temp)):
            if ((coefficient * j) >= len(temp)): break;
            freqChanged[round(coefficient * j)] += temp[j];
            
        result[i:i + batchSize] += np.fft.irfft(freqChanged) * window;
    
    return result.astype(arr.dtype)

In [216]:
applyMagic('example.wav', 0.5, 2048);

Файл записан!
